In [57]:
import torch
import pylab as plt
import numpy as np

In [241]:
def down1d(x):
    # Squeeze to make work with zero and one batch dimensions
    return torch.nn.functional.avg_pool1d(x.unsqueeze(0), 2).squeeze(0)

def down2d(x):
    return torch.nn.functional.avg_pool2d(x.unsqueeze(0), 2).squeeze(0)

def down3d(x):
    return torch.nn.functional.avg_pool3d(x.unsqueeze(0), 2).squeeze(0)

def up1d(x):
    return x.repeat_interleave(2, dim = -1)

def up2d(x):
    return x.repeat_interleave(2, dim = -2).repeat_interleave(2, dim = -1)

def up3d(x):
    return x.repeat_interleave(2, dim = -3).repeat_interleave(2, dim = -2).repeat_interleave(2, dim = -1)

def avg1d(x):
    return up1d(down1d(x))

def avg2d(x):
    return up2d(down2d(x))

def avg3d(x):
    return up3d(down3d(x))

In [242]:
def Phi1d(x):
    """
    Takes 1-dim array x of length 2^L and turns it into 1-dim array of length 2*2^L-1
    [x1, x2, x3, x4, (x1+x2)/2, (x3+x4)/2, (x1+x2+x3+x4)/4] etc
    """
    length = x.shape[-1]
    levels = int(np.log2(length))
    temp = x
    out = [temp]
    for l in range(levels):
        temp = down1d(temp)
        out.append(temp)
    return torch.cat(out, dim=-1)

def PhiT1d(x):
    """
    Transpose of Phi1d
    """
    length = int((x.shape[-1]+1)/2)
    levels = int(np.log2(length))
    tmp = x
    out = tmp[..., :length]
    for l in range(levels, 0, -1):
        tmp = torch.repeat_interleave(tmp[..., length:], 2, dim=-1)/2
        out += tmp[..., :length]
    return out

def Cont1d(x):
    """
    Calculates low-res contrastive vector.
    """
    L = int((x.shape[-1]+1)/2)
    x = torch.repeat_interleave(x[..., L:], 2, dim=-1)
    x = torch.cat([x, x[..., -1:]*0], dim = -1)
    return x

In [243]:
x = torch.randn(1, 256)*0 + 1
print(x)
y = Phi1d(x)
z = Cont1d(y)
#z = torch.repeat_interleave(y[..., 4:], 2, dim=-1)
#z = torch.cat([z, z[..., -1:]*0], dim = -1)
PhiT1d(y)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,

tensor([[1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961,
         1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961,
         1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961,
         1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961,
         1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961,
         1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961,
         1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961,
         1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961,
         1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961,
         1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961,
         1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961,
         1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961, 1.9961,
         1.9961, 1.9961, 1.9